In [ ]:
conda install -c conda-forge ta-lib

In [ ]:
# !pip uninstall numpy pandas -y

In [ ]:
conda install numpy=1.21 pandas=1.3

In [ ]:
!pip install pandas-ta tuneta awswrangler 

In [ ]:
!find ~/.cache/pip -type d -name "__pycache__" -exec rm -r {} +

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tuneta.tune_ta import TuneTA
import awswrangler as wr
import pandas_ta as pta
import talib as tta

In [ ]:
model_data = pd.read_parquet(r"s3://sg-ds-project/alpha-generator/training_data_transform/model_data_2005_20240ct.parquet")

In [ ]:
pd.set_option('display.max_columns',None)
model_data.head(3)

In [ ]:
model_data = model_data[model_data['date']>='2018-01-01']

In [ ]:
model_data.shape

In [ ]:
model_data['date'].min(),model_data['date'].max()

In [ ]:
model_data['nse_symbol'].nunique()

In [ ]:
pd.set_option('display.max_columns',None)
model_data.head(1)

In [ ]:
import os
os.cpu_count()

In [ ]:
model_data['date'] = pd.to_datetime(model_data['date'])

train_df = model_data[model_data['date'] < '2023-01-01']
test_df = model_data[model_data['date'] >= '2023-01-03']

train_df['date'] = pd.to_datetime(train_df['date'])

if 'nse_symbol' in train_df.index.names:
    train_df.reset_index(inplace=True, drop=False)  # Use drop=False to prevent dropping the existing index

if 'level_0' in train_df.columns:
    train_df.drop(columns='level_0', inplace=True)

train_df.set_index('date', inplace=True)

assert pd.api.types.is_datetime64_any_dtype(train_df.index), "Date index is not of type datetime"

print(train_df.index)
print(train_df.index.dtype)

# Initialize and fit TuneTA (now the index is only the datetime 'date')
tt = TuneTA(n_jobs=40, verbose=True)
tt.fit(train_df[['open', 'high', 'low', 'close', 'volume']], train_df['target_low_intraday'],
    indicators=[
#                 'tta.SMA', 'tta.EMA', 'tta.DEMA', 
#                  'tta.ADX', 'tta.MACD', 'tta.KAMA', 'tta.BBANDS', 'tta.RSI', 'tta.MFI', 'tta.CCI', 'pta.apo', 'pta.ao', 'pta.adosc', 'pta.aberration', 'pta.accbands',
#                  'pta.aobv', 'pta.amat', 'tta.ROC', 'pta.alma', 'tta.AROON', 'pta.bias', 'pta.cg', 'pta.atr', 'pta.cmf', 
#                 'pta.cdl', 'pta.cti', 'pta.coppock', 'pta.cksp', 'pta.chop', 'pta.brar',
#                  'pta.cmo', 'pta.eri', 'pta.dm', 'pta.donchian', 'pta.fwma', 'pta.fisher', 'pta.efi', 'pta.eom',
#                  'pta.hilo', 'pta.entropy', 'pta.er', 'pta.ebsw', 'pta.natr', 'pta.kst', 'pta.kc', 'pta.massi',
#                  'pta.psar', 'pta.percent_return', 'pta.ichimoku', 'pta.ppo', 'pta.log_return', 'pta.median',
#                  'pta.pwma', 'pta.hma', 'pta.linreg', 'pta.pvo', 'pta.kdj', 'pta.pgo', 'pta.increasing', 'pta.mad',
#                  'pta.kurtosis', 'pta.mom', 'pta.pdist', 'pta.psl', 'pta.inertia', 'pta.vidya', 'pta.trix',
#                  'pta.supertrend', 'pta.sinwma', 'pta.trima', 'pta.swma', 'pta.zlma', 'pta.vwma', 'pta.tema',
#                  'pta.ssf', 'pta.t3', 'pta.tsi', 'pta.zscore', 'pta.stdev', 'pta.vhf',
#                  'pta.squeeze_pro','pta.slope', 'pta.ttm_trend', 'pta.willr', 'pta.uo', 'pta.vortex',
#                  'pta.stochrsi', 'pta.skew', 'pta.thermo',
#                  'pta.qstick', 'pta.rsx', 'pta.rvi', 'pta.qqe', 'pta.rvgi'
#         'pta.cdl_z', 'pta.decreasing', 'pta.dema', 'pta.ema','pta.dpo'
#         'pta.pvi','pta.squeeze',
               ],
    ranges=[(3, 30)],
    trials=100,
    early_stop=20,
)

In [ ]:
tt.fit_times()

In [ ]:
tt.report(target_corr=True, features_corr=False)

In [ ]:
# Indicator Correlation to Target:

#                           Correlation
# ----------------------  -------------
# tta_SMA_timeperiod_42                                                        0.067251
# tta_EMA_timeperiod_42                                                        0.066128
# tta_SMA_timeperiod_13                                                        0.065421
# tta_DEMA_timeperiod_18                                                       0.06539
# tta_EMA_timeperiod_8                                                         0.065039
# tta_DEMA_timeperiod_38                                                       0.06398
# pta_bias_length_27_mamode_sma                                                0.107053
# tta_ROC_timeperiod_27                                                        0.103258
# pta_cg_length_8                                                              0.092069
# tta_RSI_timeperiod_22                                                        0.080802
# tta_MFI_timeperiod_25                                                        0.079108
# tta_ADX_timeperiod_22                                                        0.078343
# pta_cmf_length_22                                                            0.073775
# pta_accbands_length_5_mamode_dema                                            0.073053
# pta_alma_length_22                                                           0.070534
# pta_aberration_length_29_atr_length_15                                       0.070119
# tta_KAMA_timeperiod_27                                                       0.069833
# tta_BBANDS_timeperiod_19                                                     0.066345
# pta_amat_fast_24_slow_25_lookback_28_mamode_rma                              0.061859
# tta_CCI_timeperiod_25                                                        0.060559
# pta_atr_length_28_mamode_ema                                                 0.042708
# pta_adosc_fast_29_slow_28                                                    0.036085
# tta_MACD_fastperiod_21_slowperiod_28_signalperiod_8                          0.032823
# pta_ao_fast_8_slow_28                                                        0.028705
# pta_apo_fast_18_slow_10_mamode_rma                                           0.027499
# tta_AROON_timeperiod_27                                                      0.027439
# pta_aobv_fast_18_slow_29_max_lookback_22_min_lookback_22_mamode_sinwma       0.015249
# pta_natr_length_5_mamode_trima                                                     0.234931
# pta_massi_fast_5_slow_28                                                           0.111964
# pta_ppo_fast_9_slow_28_signal_24_mamode_ema                                        0.109022
# pta_coppock_length_4_fast_10_slow_26                                               0.104897
# pta_kst_roc1_11_roc2_17_roc3_9_roc4_17_sma1_7_sma2_6_sma3_6_sma4_26_signal_6       0.103894
# pta_percent_return_length_27                                                       0.103258
# pta_chop_length_11_atr_length_26                                                   0.101836
# pta_log_return_length_27                                                           0.099928
# pta_entropy_length_27                                                              0.093579
# pta_cmo_length_22                                                                  0.080802
# pta_psar                                                                           0.07666
# pta_donchian_lower_length_27_upper_length_22                                       0.073426
# pta_kc_length_10_mamode_tema                                                       0.072242
# pta_hilo_high_length_16_low_length_28_mamode_swma                                  0.070584
# pta_ichimoku_tenkan_11_kijun_24_senkou_11                                          0.07051
# pta_median_length_28                                                               0.070222
# pta_cksp_p_7_q_6                                                                   0.069216
# pta_fwma_length_24                                                                 0.068811
# pta_fisher_length_27_signal_5                                                      0.055821
# pta_er_length_21                                                                   0.054047
# pta_brar_length_25                                                                 0.053679
# pta_cti_length_25                                                                  0.051336
# pta_efi_length_7_mamode_rma                                                        0.040381
# pta_dm_length_28_mamode_ema                                                        0.032617
# pta_eri_length_25                                                                  0.023459
# pta_ebsw_length_23                                                                 0.021658
# pta_eom_length_26                                                                  0.015076
# pta_cdl                                                                            0.0027
# pta_trix_length_27_signal_5                        0.10246
# pta_pvo_fast_3_slow_29_signal_26                   0.100062
# pta_tsi_fast_27_slow_30_signal_27_mamode_rma       0.078732
# pta_pgo_length_26                                  0.07645
# pta_vidya_length_4                                 0.074813
# pta_supertrend_length_13                           0.070753
# pta_pwma_length_28                                 0.070249
# pta_swma_length_28                                 0.070119
# pta_trima_length_28                                0.070119
# pta_sinwma_length_28                               0.070099
# pta_ssf_length_22                                  0.069594
# pta_vwma_length_22                                 0.069458
# pta_hma_length_27                                  0.068603
# pta_linreg_length_25                               0.068559
# pta_zlma_length_19                                 0.068548
# pta_tema_length_7                                  0.068089
# pta_t3_length_26                                   0.06582
# pta_kdj_length_27_signal_27                        0.063131
# pta_zscore_length_25                               0.059211
# pta_increasing_length_25                           0.048164
# pta_vhf_length_14                                  0.042719
# pta_kurtosis_length_25                             0.027755
# pta_inertia_length_5_rvi_length_4                  0.02626
# pta_mom_length_25                                  0.025804
# pta_stdev_length_27_ddof_5                         0.025706
# pta_mad_length_22                                  0.024234
# pta_psl_length_6                                   0.013895
# pta_pdist                                          0.012273
# pta_qqe_length_18_smooth_26                                                            0.072137
# pta_rsx_length_25                                                                      0.059999
# pta_willr_length_27                                                                    0.047264
# pta_vortex_length_21                                                                   0.044969
# pta_ttm_trend_length_22                                                                0.044436
# pta_qstick_length_25                                                                   0.039306
# pta_squeeze_pro_bb_length_9_kc_length_15_mom_length_30_mom_smooth_29_mamode_swma       0.035038
# pta_rvgi_length_20_swma_length_11                                                      0.034898
# pta_skew_length_28                                                                     0.029961
# pta_uo_fast_19_medium_15_slow_15                                                       0.028397
# pta_slope_length_25                                                                    0.025804
# pta_rvi_length_15                                                                      0.020988
# pta_stochrsi_length_24_rsi_length_25_k_15_d_7_mamode_midpoint                          0.016
# pta_thermo_length_22_long_14_short_8_mamode_sinwma                                     0.00880
# pta_ema_length_28                0.069926
# pta_dema_length_16               0.068533
# pta_cdl_z_length_27_ddof_5       0.059821
# pta_decreasing_length_25         0.048001
# pta_dpo_length_25                0.028163